# Loading AR6 Sea Level Rise projections

In [ ]:
from dotenv import load_dotenv

load_dotenv()

import os

import fsspec
import geopandas as gpd
import hvplot.pandas
import hvplot.xarray
import numpy as np
import odc.stac
import pystac
import rioxarray
import shapely
import stac_geoparquet
import xarray as xr
from ipyleaflet import Map, basemaps

from coastpy.stac.utils import read_snapshot

sas_token = os.getenv("AZURE_STORAGE_SAS_TOKEN")
storage_options = {"account_name": "coclico", "sas_token": sas_token}

## Zoom to your area of interest

In [ ]:
m = Map(basemap=basemaps.Esri.WorldImagery, scroll_wheel_zoom=True)
m.center = m.center = (53.19, 4.9)
m.zoom = 12
m.layout.height = "800px"
m

In [ ]:
from coastpy.geo.utils import get_region_of_interest_from_map

roi = get_region_of_interest_from_map(m, default_extent=(4.757, 53.108, 5.042, 53.272))
west, south, east, north = list(roi.total_bounds)

In [ ]:
import re

import geopandas as gpd
import pandas as pd

catalog = pystac.Catalog.from_file(
    "https://coclico.blob.core.windows.net/stac/v1/catalog.json"
)
collection = catalog.get_child("slp6")
snapshot = read_snapshot(collection)


def preprocess_snapshot(df: gpd.GeoDataFrame) -> gpd.GeoDataFrame:
    """
    Add 'ssp', 'slr_ens', and 'datetime' columns parsed from the 'id' column.
    """
    pattern = r"ssp=([^/]+)/slr_ens([^/]+)/(\d{4})\.tif"

    extracted = df["id"].str.extract(pattern)
    extracted.columns = ["ssp", "slr_ens", "datetime"]
    extracted["datetime"] = pd.to_datetime(extracted["datetime"], format="%Y")

    for col in extracted.columns:
        df[col] = extracted[col]

    return df


snapshot = preprocess_snapshot(snapshot)

## Items for region of interest

In [ ]:
items_roi = gpd.sjoin(snapshot, roi).drop(columns=["index_right"])

## Sea level rise projection structure

There is one tif file per datetime, ssp and sea-level-rise ensemble. The unique values for each of these variables are: 

In [ ]:
print(items_roi["datetime"].unique())

In [ ]:
print(items_roi["slr_ens"].unique())

In [ ]:
print(items_roi["ssp"].unique())

## Select one item

In [ ]:
date = "2100-01-01 00:00:00"
ssp = "2-45"
slr_ens = "50.0"

item = items_roi[
    (items_roi["datetime"] == date)
    & (items_roi["ssp"] == ssp)
    & (items_roi["slr_ens"] == slr_ens)
]

print(item.shape)

## Read this item

This fails because the data is in a locked container or doesn't exist. 

In [ ]:
item_collection = stac_geoparquet.to_item_collection(item)
odc.stac.load(item_collection)